In [3]:
!pip install pulp
from pulp import *

In [26]:
# Creates a list of the products
media = ['TV', 'Print', 'Web', 'SocialMedia']

# A dictionary of the cost per medium
cost = {'TV': 500, 
         'Print': 200,
         'Web': 250,
         'SocialMedia': 125}

# A dictionary of the reach per medium
reach = {'TV': 50, 
         'Print': 25,
         'Web': 20,
         'SocialMedia': 15}

# A dictionary of the max ads per medium [increased to facilitate comparison]
maxAds = {'TV': 20, 
         'Print': 15,
         'Web': 20,
         'SocialMedia': 30}

# A dictionary of whether or not a medium is digital
channelType = {'TV': 1, 
         'Print': 1,
         'Web': 0,
         'SocialMedia': 0}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Media Planning", LpMaximize)

# A dictionary called 'production_vars' is created to contain the referenced Variables
x = LpVariable.dicts("x",media,0)

z = LpVariable('z',lowBound = 0, cat='Continuous')

# Create objective function
prob += z, "obj_reach"

# Create cost constraint
prob += lpSum([x[i]*cost[i] for i in media]) <= 13875, "Total Cost of Media"

# reach Classic
prob += lpSum([x[i]*reach[i]*channelType[i] for i in media]) >= z, "ReachClassic"

# reach Digital
prob += lpSum([x[i]*reach[i]*(1-channelType[i]) for i in media]) >= z, "ReachDigital"


# Create max constraints
for m in media:
    prob += x[m] <= maxAds[m], "{}".format(m)

In [27]:
print(prob)

Media Planning:
MAXIMIZE
1*z + 0
SUBJECT TO
Total_Cost_of_Media: 200 m_Print + 125 m_SocialMedia + 500 m_TV + 250 m_Web
 <= 13875

ReachClassic: 25 m_Print + 50 m_TV - z >= 0

ReachDigital: 15 m_SocialMedia + 20 m_Web - z >= 0

TV: m_TV <= 20

Print: m_Print <= 15

Web: m_Web <= 20

SocialMedia: m_SocialMedia <= 30

VARIABLES
m_Print Continuous
m_SocialMedia Continuous
m_TV Continuous
m_Web Continuous
z Continuous



In [28]:
# Optimize

prob.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])

# Print the value of the variables at the optimum
for i in media:
    print("%s = %f" % (x[i].name, x[i].varValue))

# Print the value of the objective
print("Objective = %f" % value(prob.objective))


# Print Classic Reach
val = 0
for i in media:
    val = val + x[i].varValue * reach[i] * channelType[i]
print("Classic Reach = %f" %val)

# Print Digital Reach
val2 = 0
for i in media:
    val2 = val2 + x[i].varValue * reach[i] * (1-channelType[i])
print("Digital Reach = %f" %val2)

#Print Total Reach for comparison with before
print("Total Reach = %f" %(val2+val))

Status = Optimal
m_TV = 7.166667
m_Print = 15.000000
m_Web = 14.166667
m_SocialMedia = 30.000000
Objective = 733.333330
Classic Reach = 733.333335
Digital Reach = 733.333340
Total Reach = 1466.666675
